In [2]:
from RADARDataCapture import FMCWRADARDataCapture
from MOCAPDataCapture import MOCAPDataCapture
from FPDataCapture import FPDataCapture
import h5py
import os
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
from datetime import datetime
import statistics
from matplotlib import animation

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
ACTUATOR_BUFFER_TIME = 2.75

# Load the CSV file with the structure described
csv_path = "/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/radar_seconds_per_frame_t0_tf.csv" 
df_tf = pd.read_csv(csv_path)

print(df_tf.head())

# Define the lists of parameters
participants = ['01', '02', '03', '04', '05', '08', '10', '12', '13', '14', '15', '16', '18', '22', '24']
# participants = ['04']
movements = ["MNTRL", "MNTRR"]
versions = ["V1", "V2", "V3"]

# Placeholder for the base file path, adjust as necessary for your environment
base_file_path_template = '/Volumes/FourTBLaCie/Dan_Yoga_Study/Data/{participant}/{participant}_{movement}_MC_{version}.tsv'

# Initialize a list to collect data
data_for_csv = []

# Problem FP Captures list
problem_fp_captures = [] 

# Loop over each combination of participant, movement, and version
for participant in participants:
    for movement in movements:
        for version in versions:
            # Construct the RADAR capture identifier to match with the CSV data
            RADAR_capture = f"{participant}_{movement}_RR_{version}"
            
            if RADAR_capture == "12_MNTRL_RR_V1":
                continue
            
            # Extract MOCAP_End_Time for the current RADAR_capture
            if RADAR_capture in df_tf['RADAR_capture'].values:
                end_time = np.round(df_tf.loc[df_tf['RADAR_capture'] == RADAR_capture, 'MOCAP_End_Time'].iloc[0] - ACTUATOR_BUFFER_TIME,2)
            else:
                end_time = None  # Handle case where RADAR_capture is not found in the CSV
                
            # Construct the file path
            base_file_path = base_file_path_template.format(participant=participant, movement=movement, version=version)
            
            try:
                fp_data_capture = FPDataCapture(base_file_path)
                print(RADAR_capture)
                lift_times, down_times = fp_data_capture.identify_foot_lift()        
                data_for_csv.append([RADAR_capture, lift_times, down_times, end_time])
                
                if len(lift_times)!= 3 or len(down_times) != 2 or end_time == None:
                    problem_fp_captures.append(RADAR_capture)
                
            except FileNotFoundError:
                # Handle the case where the file does not exist or an error occurs in data processing
                print(f"File not found or error in processing for {base_file_path}")


# Convert the list of data into a DataFrame
output_df = pd.DataFrame(data_for_csv, columns=['RADAR_capture', 'Lift_Times', 'Down_Times', 'MOCAP_End_Time'])

# Define the output CSV file path
output_csv_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/all_lift_down_times_v3.csv'

# Save the DataFrame to CSV
output_df.to_csv(output_csv_path, index=False)

print(output_df.head())  # Display the first few rows of the DataFrame for verification

print(f"Data saved to {output_csv_path}")
print(f"Problems with force plate captures: {problem_fp_captures}")

    RADAR_capture  Seconds_per_Frame  MOCAP_Start_Time  MOCAP_End_Time  \
0  01_MNTRL_RR_V1           0.036454              1.86           33.32   
1  01_MNTRL_RR_V2           0.036464              1.90           33.15   
2  01_MNTRL_RR_V3           0.036547              1.92           34.63   
3  01_MNTRR_RR_V1           0.036467              1.91           32.98   
4  01_MNTRR_RR_V2           0.036370              1.95           33.41   

   RADAR_Start_Frame  RADAR_End_Frame  
0                 44              907  
1                 41              898  
2                 43              938  
3                 45              897  
4                 41              906  
01_MNTRL_RR_V1


Combine CSVs

In [6]:
# Paths to the CSV files
csv_path1 = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/all_lift_down_times_edited.csv'   
csv_path2 = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/radar_seconds_per_frame_t0_tf.csv'  

# Load the CSVs into DataFrames
df_lift_down_times = pd.read_csv(csv_path1)
df_radar_info = pd.read_csv(csv_path2)

# # Remove the 'RADAR_End_Frame' column from df_radar_info
# df_radar_info = df_radar_info.drop('RADAR_End_Frame', axis=1)
df_radar_info = df_radar_info.drop('MOCAP_End_Time', axis=1)

# Merge the DataFrames on 'RADAR_capture'
merged_df = pd.merge(df_lift_down_times, df_radar_info, on='RADAR_capture')

merged_df['RADAR_End_Frame'] = (merged_df['RADAR_Start_Frame'] + (merged_df['MOCAP_End_Time']-merged_df['MOCAP_Start_Time'])/merged_df['Seconds_per_Frame']).astype('int')

# Optionally, save the merged DataFrame to a new CSV file
output_csv_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/FP_RADAR_FULL_DATA2.csv'  # Update this path accordingly
merged_df.to_csv(output_csv_path, index=False)

print("Merging completed and saved to:", output_csv_path)


Merging completed and saved to: /Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/FP_RADAR_FULL_DATA2.csv


Generate Foot Lift Foot Down Pairs

In [5]:
import ast  # Safe evaluation of strings containing Python expressions

# Load the merged CSV
csv_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/FP_RADAR_FULL_DATA.csv'  # Update this path
df = pd.read_csv(csv_path)

# Initialize the list to store the structured data
structured_data = []

# Process each row in the DataFrame
for index, row in df.iterrows():
    # Safe evaluation of the lift and down times from string to list
    lift_times = ast.literal_eval(row['Lift_Times'])
    down_times = ast.literal_eval(row['Down_Times']) + [row['MOCAP_End_Time']]
    
    for i, lift_time in enumerate(lift_times):
        # Use the corresponding down time or MOCAP_End_Time for the last lift time
        down_time = down_times[i] if i < len(down_times) else row['MOCAP_End_Time']
        
        # Calculate START_FRAME and END_FRAME for each interval
        start_frame = int(((lift_time - row['MOCAP_Start_Time']) / row['Seconds_per_Frame']) + row['RADAR_Start_Frame'])
        end_frame = int(((down_time - row['MOCAP_Start_Time']) / row['Seconds_per_Frame']) + row['RADAR_Start_Frame'])
        
        if down_time > 29:
            is_final_tx = True
        else:
            is_final_tx = False
        
        # Append the structured data
        structured_data.append({
            'RADAR_capture': row['RADAR_capture'],
            'Start_End_Times': (lift_time, down_time),
            'Start_End_Frames': (start_frame, end_frame),
            'tx':(i+1),
            'is_final_tx': (is_final_tx)
            
        })

# Create a new DataFrame from the structured data
new_df = pd.DataFrame(structured_data)

print(new_df.head())

# Specify the new CSV file name to save the updated DataFrame
new_csv_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/FP_RADAR_START_END_TIME_FRAMES.csv'

# Save to CSV file, without the index
new_df.to_csv(new_csv_path, index=False)

print(f'Data saved to {new_csv_path}')

    RADAR_capture    Start_End_Times Start_End_Frames  tx  is_final_tx
0  01_MNTRL_RR_V1  (12.60667, 15.05)       (338, 405)   1        False
1  01_MNTRL_RR_V1     (18.25, 22.83)       (493, 619)   2        False
2  01_MNTRL_RR_V1     (26.24, 30.57)       (712, 831)   3         True
3  01_MNTRL_RR_V2     (10.48, 15.05)       (276, 401)   1        False
4  01_MNTRL_RR_V2     (18.41, 22.86)       (493, 615)   2        False
Data saved to /Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/FP_RADAR_START_END_TIME_FRAMES.csv
